# Aggregate sample point neighborhood indicators to generate output indicators for within and accross-city comparison
Description: This script is for preparing all the fields for within and accross city indicators
This script should be run after the sample point geopackages prepared for all cities (sp.py), then use this is script to get all the final output

**Two outputs:**
1. global_indicators_hex_250m.gpkg  

|indicators | data type | description |
|---- | --- | --- |
|urban_sample_point_count | int | Count of urban sample points associated with each hexagon (judge using intersect); this must be positive.  Zero sample count hexagons are not of relevance for output |
|pct_access_500m_supermarkets | float | Percentage of sample points with pedestrian network access to supermarkets within (up to and including) 500 metres |
|pct_access_500m_covenience | float | Percentage of sample points with pedestrian network access to convenience within (up to and including) 500 metres |
|pct_access_500m_pt_any | float | Percentage of sample points with pedestrian network access to public transport within (up to and including) 500 metres |
|pct_access_500m_public_open_space | float | Percentage of sample points with pedestrian network access to public open space within (up to and including) 500 metres |
|local_nh_population_density | float | Average local neighbourhood population density for sample points |
|local_nh_intersection_density | float | Average local neighbourhood intersection density for sample points |
|local_daily_living | float | Average daily living score for sample points |
|local_walkability | float | Average walkability score for sample points (calculated relative to all sample points across city) |
|all_cities_z_nh_population_density | float | Z-score of local neighbourhood population density for hexagons relative to all cities |
|all_cities_z_nh_intersection_density | float | Z-score of local neighbourhood intersection density for hexagons relative to all cities |
| all_cities_z_daily_living | float | Z-score of daily living score for hexagons relative to all cities |
| all_cities_walkability | float | Walkability index for hexagons relative to all cities |


2. global_indicators_city.gpkg  

|indicators | data type | description |
|---- | --- | --- |
|pop_pct_access_500m_supermarkets | float | Percentage of population with access to supermarkets within (up to and including) 500 metres|
|pop_pct_access_500m_convenience | float | Percentage of population with pedestrian network access to convenience within (up to and including) 500 metres |
|pop_pct_access_500m_pt_any | float | Percentage of population with pedestrian network access to public transport within (up to and including) 500 metres |
|pop_pct_access_500m_public_open_space | float | Percentage of population with pedestrian network access to public open space within (up to and including) 500 metres |
|pop_nh_pop_density | float | Average local neighbourhood population density for population |
| pop_nh_intersection_density | float | Average local neighbourhood intersection density for population |
| pop_daily_living | float | Average daily living score for population |
| pop_walkability | float | Average walkability index for population (within city) |
| all_cities_pop_z_nh_population_density | float | Z-score of population weighted neighbourhood population density for city relative to all cities |
| all_cities_pop_z_nh_intersection_density | float | Z-score of population weighted neighbourhood intersection density for city relative to all cities |
| all_cities_pop_z_daily_living | float | Z-score of population weighted daily living score for city relative to all cities |
|all_cities_walkability | float | Walkability index for city relative to all cities|

In [4]:
import json
import os
import time
import pandas as pd
import geopandas as gpd
import sys
from setup_aggr import * #module for all aggregation functions used in this notebook

In [5]:
# get the work directory
dirname = os.path.dirname('')
jsonFile = "./configuration/" + sys.argv[1]
jsonPath = os.path.join(dirname, jsonFile)

#read configeration file for all cities (here only include vic and valencia for testing purposes)
with open("configuration/cities2.json") as json_file:
    config = json.load(json_file)

# specify input and outpur location
folder = config['folder']
input_folder = config['input_folder']

# read city names from json
cites = list(config['cityNames'].values())
print("Cities:{}".format(cites))

Cities:['valencia', 'vic']


## 1. Generate "global_indicators_hex_250m.gpkg" for within-city hex-level indicators

In [6]:
# create the output path of "global_indicators_hex_250m.gpkg"
gpkgOutput_hex250 = os.path.join(dirname, folder,
                                 config['output_hex_250m'])

# read the input path of pre-prepared sample point of each city
gpkgInput_ori = []
for gpkg in list(config['gpkgNames'].values()):
    gpkgInput_ori.append(os.path.join(dirname, input_folder, gpkg))

print("City-specific input datasources with sample point stats are from: ",  gpkgInput_ori)
  
print("All within-city indicators will be saved to: ", gpkgOutput_hex250)

City-specific input datasources with sample point stats are from:  ['data/input/valencia_es_2019_1600m_buffer.gpkg', 'data/input/vic_es_2019_1600m_buffer.gpkg']
All within-city indicators will be saved to:  data/output/test_global_indicators_hex_250m.gpkg


In [7]:
# prepare aggregation for hexes across all cities
print("Calculate hex-level indicators weighted by sample points within each city:")

cities_hexes = []
for index, gpkgInput in enumerate(gpkgInput_ori):
    print(index, cites[index])
    city_hexes = calc_hexes_pct_sp_access(gpkgInput, gpkgOutput_hex250, cites[index],
               config['samplepointResult'], config['hex250'], config)
    cities_hexes.append(city_hexes)
    

Calculate hex-level indicators weighted by sample points within each city:
0 valencia
1 vic


In [8]:
cities_hexes[0].head(3)

,index,geometry,urban_sample_point_count,pct_access_500m_supermarkets,pct_access_500m_convenience,pct_access_500m_pt_any,pct_access_500m_public_open_space,local_nh_population_density,local_nh_intersection_density,local_daily_living,local_walkability,study_region
12,14290,"MULTIPOLYGON (((729706.000 4363806.000, 729768...",12,0.0,33.333333,83.333333,100.0,5863.539898,44.837584,1.166667,-5.963264,valencia
30,16525,"MULTIPOLYGON (((728962.000 4366858.000, 729024...",14,100.0,100.000000,100.000000,100.0,9505.956210,58.387891,3.000000,-2.843656,valencia
35,17173,"MULTIPOLYGON (((725614.000 4367730.000, 725676...",26,0.0,100.000000,100.000000,100.0,11393.625880,94.476445,2.000000,-2.477176,valencia


In [9]:
# prepare all_cities level fields for every hex across all cities
print("Calculate hex-level indicators zscores within each city.\n" 
      "Check", [gpkgOutput_hex250], "for output indicators of each city" )

calc_hexes_zscore_walk(gpkgOutput_hex250, cites, config)

Calculate hex-level indicators zscores within each city.
Check ['data/output/test_global_indicators_hex_250m.gpkg'] for output indicators of each city


In [10]:
# check the hex layer for within-city indicators
vic_hex250 = gpd.read_file(gpkgOutput_hex250, layer=config["cityNames"]['vic'])
vic_hex250.columns

Index(['index', 'local_daily_living', 'local_nh_intersection_density',
       'local_nh_population_density', 'local_walkability',
       'pct_access_500m_convenience', 'pct_access_500m_pt_any',
       'pct_access_500m_public_open_space', 'pct_access_500m_supermarkets',
       'study_region', 'urban_sample_point_count',
       'all_cities_z_nh_population_density',
       'all_cities_z_nh_intersection_density', 'all_cities_z_daily_living',
       'all_cities_walkability', 'geometry'],
      dtype='object')

## 2. Generate "global_indicators_city.gpkg" for accross-city city-level indicators

In [11]:
# create the path of "global_indicators_city.gpkg"
gpkgOutput_cities = os.path.join(dirname, folder,
                                 config['global_indicators_city'])
print("All accross-city indicators will be saved to: ", gpkgOutput_cities)

All accross-city indicators will be saved to:  data/output/test_global_indicators_city.gpkg


In [12]:
# prepare aggregation for study region across all cities
print("Calculate city-level indicators weighted by city population:")

cities_all = []
for index, gpkgInput in enumerate(gpkgInput_ori):
    print(index, cites[index])
    city = calc_cities_pop_pct_indicators(gpkgOutput_hex250, cites[index], gpkgInput, config,
              gpkgOutput_cities)
    cities_all.append(city)


Calculate city-level indicators weighted by city population:
0 valencia
1 vic


In [13]:
cities_all[0] # index 0 is city of Valencia

,Study region,geometry,urban_sample_point_count,pop_pct_access_500m_supermarkets,pop_pct_access_500m_convenience,pop_pct_access_500m_pt_any,pop_pct_access_500m_public_open_space,pop_nh_pop_density,pop_nh_intersection_density,pop_daily_living,pop_walkability,all_cities_pop_z_daily_living,all_cities_walkability
0,Valencia,"MULTIPOLYGON (((729184.362 4381327.027, 729178...",73653,60.984933,47.814282,83.279347,89.25162,15552.28365,129.322474,1.920786,-0.622679,0.505169,1.826356


In [14]:
# check the hex layer for within-city indicators
vic_city = gpd.read_file(gpkgOutput_cities, layer=config["cityNames"]['vic'])
vic_city.columns

Index(['city', 'area_sqkm', 'urban_sample_point_count',
       'pop_pct_access_500m_supermarkets', 'pop_pct_access_500m_convenience',
       'pop_pct_access_500m_pt_any', 'pop_pct_access_500m_public_open_space',
       'pop_nh_pop_density', 'pop_nh_intersection_density', 'pop_daily_living',
       'pop_walkability', 'all_cities_pop_z_daily_living',
       'all_cities_walkability', 'geometry'],
      dtype='object')